<a href="https://colab.research.google.com/github/Arimoro2020/food-sales-predictions./blob/main/Using_Linear_Regression_to_create_a_model_to_predict_sales_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Libraries

In [19]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

Load Cleaned Data & Check first five Rows

In [20]:
df_sales = pd.read_excel('/content/drive/MyDrive/PTDataScience/Introduction_to_ML/cleansed data_sales pridiction.xlsx')
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052


Check Data Types

In [21]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 732.6+ KB


Drop Item Identifier Column because it has mostly unique values

In [22]:
df_sales.drop(columns='Item_Identifier', inplace=True)
df_sales.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052


Convert Outlet Establishment Year to Object type

In [23]:
df_sales['Outlet_Establishment_Year'] = df_sales['Outlet_Establishment_Year'].astype(str)
df_sales['Outlet_Establishment_Year']

0       1999
1       2009
2       1999
3       1998
4       1987
        ... 
8518    1987
8519    2002
8520    2004
8521    2009
8522    1997
Name: Outlet_Establishment_Year, Length: 8523, dtype: object

**Transform Categorical Columns into Dummy Variables**

1. Item Fat Content

In [24]:
item_fat_content = pd.get_dummies(df_sales['Item_Fat_Content'], drop_first=True)
df_sales = pd.concat([df_sales, item_fat_content], axis=1)
df_sales.drop(columns='Item_Fat_Content', inplace=True)
df_sales.head()

,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Regular
0,9.30,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380,0
1,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228,1
2,17.50,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700,0
3,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800,1
4,8.93,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052,0


2. Item Type

In [25]:
item_type = pd.get_dummies(df_sales['Item_Type'], drop_first=True)
df_sales = pd.concat([df_sales, item_type], axis=1)
df_sales.drop(columns='Item_Type', inplace=True)
df_sales.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Regular,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods
0,9.30,0.016047,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,5.92,0.019278,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,17.50,0.016760,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,19.20,0.000000,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


3. Outlet Identifier

In [26]:
outlet_identifier = pd.get_dummies(df_sales['Outlet_Identifier'], drop_first=True)
df_sales = pd.concat([df_sales, outlet_identifier], axis=1)
df_sales.drop(columns='Outlet_Identifier', inplace=True)
df_sales.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Regular,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods,OUT013,OUT017,OUT018,OUT019,OUT027,OUT035,OUT045,OUT046,OUT049
0,9.30,0.016047,249.8092,1999,Tier 1,Supermarket Type1,3735.1380,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,2009,Tier 3,Supermarket Type2,443.4228,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,1999,Tier 1,Supermarket Type1,2097.2700,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,1998,Tier 3,Grocery Store,732.3800,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,1987,Tier 3,Supermarket Type1,994.7052,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


4. Outlet Establishment Year

In [27]:
outlet_est_year = pd.get_dummies(df_sales['Outlet_Establishment_Year'], drop_first=True)
df_sales = pd.concat([df_sales, outlet_est_year], axis=1)
df_sales.drop(columns='Outlet_Establishment_Year', inplace=True)
df_sales.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Regular,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods,OUT013,OUT017,OUT018,OUT019,OUT027,OUT035,OUT045,OUT046,OUT049,1987,1997,1998,1999,2002,2004,2007,2009
0,9.30,0.016047,249.8092,Tier 1,Supermarket Type1,3735.1380,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,5.92,0.019278,48.2692,Tier 3,Supermarket Type2,443.4228,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,17.50,0.016760,141.6180,Tier 1,Supermarket Type1,2097.2700,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,19.20,0.000000,182.0950,Tier 3,Grocery Store,732.3800,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,8.93,0.000000,53.8614,Tier 3,Supermarket Type1,994.7052,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


5. Outlet Location Type

In [28]:
loc_type = pd.get_dummies(df_sales['Outlet_Location_Type'], drop_first=True)
df_sales = pd.concat([df_sales, loc_type], axis=1)
df_sales.drop(columns='Outlet_Location_Type', inplace=True)
df_sales.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Type,Item_Outlet_Sales,Regular,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods,OUT013,OUT017,OUT018,OUT019,OUT027,OUT035,OUT045,OUT046,OUT049,1987,1997,1998,1999,2002,2004,2007,2009,Tier 2,Tier 3
0,9.30,0.016047,249.8092,Supermarket Type1,3735.1380,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
1,5.92,0.019278,48.2692,Supermarket Type2,443.4228,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,17.50,0.016760,141.6180,Supermarket Type1,2097.2700,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,19.20,0.000000,182.0950,Grocery Store,732.3800,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,8.93,0.000000,53.8614,Supermarket Type1,994.7052,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


6. Outlet Type

In [29]:
outlet_type = pd.get_dummies(df_sales['Outlet_Type'], drop_first=True)
df_sales = pd.concat([df_sales, outlet_type], axis=1)
df_sales.drop(columns='Outlet_Type', inplace=True)
df_sales.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Regular,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods,OUT013,OUT017,OUT018,OUT019,OUT027,OUT035,OUT045,OUT046,OUT049,1987,1997,1998,1999,2002,2004,2007,2009,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3
0,9.30,0.016047,249.8092,3735.1380,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,5.92,0.019278,48.2692,443.4228,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
2,17.50,0.016760,141.6180,2097.2700,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
3,19.20,0.000000,182.0950,732.3800,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,8.93,0.000000,53.8614,994.7052,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0


In [30]:

y = df_sales['Item_Outlet_Sales'].values
X = df_sales.loc[ : ,['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Regular', 'Breads', 'Breakfast', 'Canned', 'Dairy', 'Frozen Foods',
       'Fruits and Vegetables', 'Hard Drinks', 'Health and Hygiene',
       'Household', 'Meat', 'Others', 'Seafood', 'Snack Foods', 'Soft Drinks',
       'Starchy Foods', 'OUT013', 'OUT017', 'OUT018', 'OUT019', 'OUT027',
       'OUT035', 'OUT045', 'OUT046', 'OUT049', '1987', '1997', '1998', '1999',
       '2002', '2004', '2007', '2009', 'Tier 2', 'Tier 3', 'Supermarket Type1',
       'Supermarket Type2', 'Supermarket Type3']].values 



Import Train Test Split from Sklearn

In [31]:
from sklearn.model_selection import train_test_split

Split Data into Training & Testing Data

In [32]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

Import StandardScaler for Normalization

In [33]:
from sklearn.preprocessing import StandardScaler

Create Instance of Standardscaler, Fit & Transform on X

In [34]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Import LASSO from sklearn

In [35]:
from sklearn.linear_model import Lasso

Create & Train Model

In [36]:
lin = Lasso(alpha=16)
lin.fit(X_train, y_train)

Lasso(alpha=16, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

Test Model

In [37]:
pred = lin.predict(X_test)

Import Metrics for Evaluating Model

In [38]:
from sklearn import metrics
metrics.mean_squared_error(y_test, pred)

1272545.3248274017

In [39]:
RMSE = np.sqrt(metrics.mean_squared_error(y_test, pred))
RMSE

1128.0715069654946

Get Coefficients

In [44]:
MAE = metrics.mean_absolute_error(y_test, pred)
MAE

835.6534276199948

In [40]:
lin.coef_

array([   0.        ,   -5.94124716,  961.56269596,   24.38867929,
          0.        ,    0.        ,    0.        ,   -0.        ,
         -0.        ,    0.        ,   -0.        ,    0.        ,
         -1.29467499,    0.        ,    0.        ,   15.0661019 ,
          0.        ,   -0.        ,   -0.        ,    0.        ,
          0.        ,   -0.        , -384.05629849,  470.49924988,
         10.5673449 ,   -8.36020833,   -0.        ,    5.93622312,
          0.        ,   -0.        , -400.97193639,    4.12026811,
         -0.        ,    4.03212638,    8.1495466 ,   -0.        ,
          0.        ,   -0.        ,  132.8097521 ,   -0.        ,
         50.84123605])

Get Intercept

In [41]:
lin.intercept_

2181.44314562521

Which features are most associated with higher predicted sales?

In [42]:
data = {'coeff':[   0.        ,   -5.79599387,  957.9244608 ,   24.30924552,
          0.        ,    0.        ,    0.        ,   -0.        ,
         -0.        ,    0.        ,   -0.        ,    0.        ,
         -1.41444939,    0.        ,    0.        ,   15.0765633 ,
          0.        ,   -0.        ,   -0.        ,    0.        ,
          0.        ,   -0.        , -382.02185914,  465.15921503,
         10.48729989,   -8.56467987,   -0.        ,    5.8567923 ,
          0.        ,   -0.        , -397.09573814,    4.05544097,
         -0.        ,    3.95147079,    7.97430924,   -0.        ,
          0.        ,   -0.        ,  133.01808737,   -0.        ,
         49.43478712],
        'variables': ['Item_Weight', 'Item_Visibility', 'Item_MRP',
        'Regular', 'Breads', 'Breakfast', 'Canned', 'Dairy',
        'Frozen Foods', 'Fruits and Vegetables', 'Hard Drinks',
        'Health and Hygiene', 'Household', 'Meat', 'Others', 'Seafood',
        'Snack Foods', 'Soft Drinks', 'Starchy Foods', 'OUT013', 'OUT017',
        'OUT018', 'OUT019', 'OUT027', 'OUT035', 'OUT045', 'OUT046', 'OUT049',
        '1987', '1997', '1998', '1999', '2002', '2004', '2007', '2009',
        'Tier 2', 'Tier 3', 'Supermarket Type1','Supermarket Type2', 'Supermarket Type3'] }       


df = pd.DataFrame(data, columns=['coeff', 'variables'])    
df                


,coeff,variables
0,0.000000,Item_Weight
1,-5.795994,Item_Visibility
2,957.924461,Item_MRP
3,24.309246,Regular
4,0.000000,Breads
5,0.000000,Breakfast
6,0.000000,Canned
7,-0.000000,Dairy
8,-0.000000,Frozen Foods
9,0.000000,Fruits and Vegetables


Item MPR, OUT027, & Supermarket Type1 features are most associated with higher predicted sale